In [128]:
import pandas as pd
from collections import Counter

# REGISTROS DOS TESTE REALIZADOS
# teste inicial: home, busca, logado => comprou
# home, busca
# home, logado
# busca, logado
# busca: 85,71% (7 testes)
# comparando [adaboost, multinomial] usando [home, busca, logado]

df = pd.read_csv('buscas2.csv')

X_df = df[['home','busca','logado']]
Y_df = df['comprou']
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

# valores dentro dos dataframes
X = Xdummies_df.values
Y = Ydummies_df.values

porcentagem_treino = 0.8
porcentagem_teste = 0.1
tamanho_treino = int(porcentagem_treino * len(Y))
tamanho_teste = int(porcentagem_teste * len(Y))
tamanho_validacao = len(Y) - tamanho_treino - tamanho_teste

treino_dados = X[:tamanho_treino]
treino_marcacoes = Y[:tamanho_treino]

fim_treino = tamanho_treino + tamanho_teste

teste_dados = X[tamanho_treino:fim_treino]
teste_marcacoes = Y[tamanho_treino:fim_treino]

validacao_dados = X[fim_treino:]
validacao_marcacoes = Y[fim_treino:]

In [129]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes):
    modelo.fit(treino_dados, treino_marcacoes)
    
    resultado = modelo.predict(teste_dados)
    acertos = resultado == teste_marcacoes
    
    # taxa de acertos do algoritmo treinado
    total_acertos = sum(acertos)
    total_elementos = len(teste_marcacoes)
    taxa_acertos = 100.0 * total_acertos / total_elementos
    
    msg = "Taxa de acerto do algoritmo {0}: {1}".format(nome, taxa_acertos)
    print(msg)
    
    return taxa_acertos

In [130]:
def teste_real(modelo, validacao_dados, validacao_marcacoes):
    modelo.fit(validacao_dados, validacao_marcacoes)
    resultado = modelo.predict(validacao_dados)
    
    acertos = resultado == validacao_marcacoes
    total_acertos = sum(acertos)
    total_elementos = len(validacao_marcacoes)
    
    taxa_acertos = 100.0 * total_acertos / total_elementos
    
    msg = "Taxa de acerto do vencedor entre os dois algoritmos no mundo real: {0}".format(taxa_acertos)
    print(msg)

In [131]:
# treinando com MultinomialNB
from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

# treinando com AdaBoost
from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier",modelo, treino_dados, treino_marcacoes, teste_dados, teste_marcacoes)

if resultadoMultinomial > resultadoAdaBoost:
    vencedor = modeloMultinomial
else:
    vencedor = modeloAdaBoost
    
teste_real(vencedor, validacao_dados, validacao_marcacoes)

# taxa de acertos do algoritmo base
acerto_base = max(Counter(validacao_marcacoes).values())
taxa_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print('Taxa de acerto base: %f' % taxa_acerto_base)

total_elementos = len(validacao_dados)
print('Total de elementos: %d' % total_elementos)

Taxa de acerto do algoritmo MultinomialNB: 85.71428571428571
Taxa de acerto do algoritmo AdaBoostClassifier: 85.71428571428571
Taxa de acerto do vencedor entre os dois algoritmos no mundo real: 75.0
Taxa de acerto base: 62.500000
Total de elementos: 8
